In [4]:
# if used pip install package
!pip install xgboost
!pip install lightgbm
!pip install wget

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 109.7 MB 12.3 MB/s eta 0:00:01    |███▉                            | 13.1 MB 61.3 MB/s eta 0:00:02     |█████▌                          | 18.9 MB 61.3 MB/s eta 0:00:02     |████████████▎                   | 42.1 MB 61.3 MB/s eta 0:00:02          | 49.8 MB 6.9 MB/s eta 0:00:09     |█████████████████▊              | 60.7 MB 6.9 MB/s eta 0:00:08     |████████████████████████▋       | 84.6 MB 81.4 MB/s eta 0:00:01�███████▏    | 93.3 MB 81.4 MB/s eta 0:00:01
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 808 kB/s eta 0:00:01
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=6579f3858eb6dcdf56dc1348ae5a759b10362bbee3d24d6fc4d59b6fa9de98c6
  Stored in directory: /home/tione/.cache/pip/wheels/c5/7d/57/a14ab81e83a58c3ccf630b021656bf453e6182666d5

In [64]:
import os
import sys
import wget
import tarfile
import zipfile
import random

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
DDIR = "data"
UDDIR = "user_data"
RESDIR = "prediction_result"

In [30]:
UID = "user_id"

# Load data (Only once)

In [7]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [8]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [9]:
# myunzip(train_fname)
# myunzip(test_fname)

# Utils

In [36]:
def bch_rencol(values, prefix="", suffix=""):
    return list(map(lambda x: f"{prefix}"+"_".join(list(map(lambda y: str(y), x)))+f"{suffix}" 
                    if hasattr(x, "__iter__") and not isinstance(x, str) 
                    else f"{prefix}"+str(x)+f"{suffix}", values))

# Data Exploration

In [10]:
# read train data
train_click_log = pd.read_csv(f"{DDIR}/train_preliminary/click_log.csv")
train_ad = pd.read_csv(f"{DDIR}/train_preliminary/ad.csv")
# tag
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")

In [11]:
# read test data
test_click_log = pd.read_csv(f"{DDIR}/test/click_log.csv")
test_ad = pd.read_csv(f"{DDIR}/test/ad.csv")

In [12]:
train_click_log.head()

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1


In [13]:
train_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213


In [14]:
test_click_log.head()

,time,user_id,creative_id,click_times
0,20,3131989,645764,1
1,20,3131989,1027422,1
2,20,3131989,1106443,1
3,20,3131989,629802,1
4,59,3131989,2839769,1


In [15]:
test_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,3,3,\N,5,598,165
2,4,4,\N,5,108,202
3,5,5,\N,5,109,6
4,6,6,\N,5,400,6


In [16]:
train_user.head()

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1


In [17]:
train_click_log.head()

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1


In [18]:
train_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213


In [19]:
train_ad.creative_id.nunique()

2481135

In [20]:
train_ad.ad_id.nunique()

2264190

# Feature engineering

In [83]:
train_feat = pd.DataFrame()
test_feat = pd.DataFrame()

In [84]:
train_feat[UID] = train_user[UID]
test_feat[UID] = np.sort(test_click_log[UID].unique())

## User Click log

In [85]:
train_click_log.head()

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1


In [86]:
# for train
tmp = train_click_log.groupby([UID]).agg(
    {
        UID: ["count"], 
        "click_times": ["sum", "max", "mean", "std"],
        "time": ["nunique", "mean", "max", "min"],
        "creative_id": ["nunique"]
    }
)

In [87]:
tmp.columns = bch_rencol(tmp.columns)
tmp.reset_index(inplace=True)
train_feat = pd.merge(train_feat, tmp, how="left", on=UID)

In [88]:
tmp = None

In [89]:
# for test
tmp = test_click_log.groupby([UID]).agg(
    {
        UID: ["count"], 
        "click_times": ["sum", "max", "mean", "std"],
        "time": ["nunique", "mean", "max", "min"],
        "creative_id": ["nunique"]
    }
)

In [90]:
tmp.columns = bch_rencol(tmp.columns)
tmp.reset_index(inplace=True)
test_feat = pd.merge(test_feat, tmp, how="left", on=UID)

In [91]:
tmp = None

In [ ]:
train_all = pd.merge(train_feat, train_user, how="left", on=UID)

In [ ]:
train_tag = train_all[[UID, "age", "gender"]]
train_feat = train_all.drop(["age", "gender"], axis=1)

# Training&Prediction

In [107]:
train_feat_tr, train_feat_val, train_tag_tr, train_tag_val = train_test_split(train_feat, train_tag, test_size=0.2)

In [108]:
lgbds_train_tr = lgb.Dataset(train_feat_tr, train_tag_tr["age"])
lgbds_train_val = lgb.Dataset(train_feat_val, train_tag_val["age"])

In [113]:
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": "10",
    "metric": "multi_error"
}

In [ ]:
model_lgb_multi = lgb.train(params, lgbds_train_tr, num_boost_round=1000, valid_sets=[lgbds_train_val], verbose_eval=50, early_stopping_rounds=100)

# Generate Prediction Result

In [39]:
init_res = pd.DataFrame(test_click_log.user_id.unique(), columns=["user_id"])

In [57]:
init_res["predicted_age"] = l

In [59]:
init_res["predicted_gender"] = l

In [58]:
l = list()
for i in range(1000000):
    l.append(random.choice(range(1,3)))

In [62]:
init_res.to_csv(f"{RESDIR}/init_res.csv", index=False)

# Cent result to COS

In [24]:
from ti import session
ti_session = session.Session()

In [63]:
inputs = ti_session.upload_data(path=f"{RESDIR}/init_res.csv", bucket="etveritas-1252104022", key_prefix=RESDIR)